##Sentiment Analysis using Vader

In [ ]:
# We will use this data- https://archive.ics.uci.edu/ml/datasets/sentiment+labelled+sentences
# Save this data in a csv and import in Python

import numpy as np
import pandas as pd
import nltk
nltk.download('all')
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#Magic command to print multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [ ]:
#Install and upgrade VADER
!pip install vaderSentiment
!pip install --upgrade vaderSentiment

In [ ]:
# In read_csv encoding cmd is fornon-ASCII characters
df = pd.read_excel("yelp_sentiment.xlsx")
df.head ()
df.shape
df.columns


,Text,Sentiment
0,!....THE OWNERS REALLY REALLY need to quit bei...,0
1,"(It wasn't busy either) Also, the building was...",0
2,(The bathroom is just next door and very nice.),1
3,* Both the Hot & Sour & the Egg Flower Soups w...,1
4,2 Thumbs Up!!,1


(996, 2)

Index(['Text', 'Sentiment'], dtype='object')

In [ ]:
df.dtypes

Text         object
Sentiment     int64
dtype: object

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    if isinstance(text, str):  # Check if the text is a string
        text = text.lower()
        # Remove special characters and digits
        text = re.sub(r'[^a-z\s]', ' ', text)
        # Remove URLs
        text = re.sub(r'http\S+|www.\S+', ' ', text)
        # Remove punctuations
        text = re.sub(r'[^\w\s]', ' ', text)
        # Remove numbers
        text = re.sub(r'\d+', ' ', text)
        # Remove extra white spaces
        text = re.sub(r'\s+', ' ', text).strip()
        # Remove @ mentions
        text = re.sub(r'@\w+', ' ', text)
        # Remove hashtags
        text = re.sub(r'#\w+', ' ', text)
        # Tokenize the text
        tokens = nltk.word_tokenize(text)
        # Remove stop words
        stop_words = set(stopwords.words('english'))
        tokens = [x for x in tokens if x not in stop_words]
        # Lemmatize the text
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(x) for x in tokens]
        # Join the tokens back into a string
        text = ' '.join(tokens)
    return text

# Apply the function to the original 'review_context' and create a new column with clean text
df['Text_Clean'] = df['Text'].apply(clean_text)
df.head()


,Text,Sentiment,Text_Clean
0,!....THE OWNERS REALLY REALLY need to quit bei...,0,owner really really need quit soooooo cheap le...
1,"(It wasn't busy either) Also, the building was...",0,busy either also building freezing cold
2,(The bathroom is just next door and very nice.),1,bathroom next door nice
3,* Both the Hot & Sour & the Egg Flower Soups w...,1,hot sour egg flower soup absolutely star
4,2 Thumbs Up!!,1,thumb


In [ ]:
# Import necessary libraries
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Instantiate the VADER sentiment analyzer
sentiment_analyzer = SentimentIntensityAnalyzer()

# Create a function to classify sentiment into positive (1) and neutral/negative (0)
def get_sentiment_label(score, threshold=0.0):
    if score > threshold:
        return 1
    else:
        return 0

df['Text'].fillna('', inplace=True)

# Check for NaN values in 'review_rating' and drop corresponding rows
df.dropna(subset=['Sentiment'], inplace=True)

# Split data into train and test sets
x = df['Text']
y = df['Sentiment']  # Assuming 'review_rating' contains the true labels
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=23)

# Optimize score threshold
best_accuracy = 0.0
best_threshold = 0.0

for threshold in [0.1 * i for i in range(-10, 11)]:
    y_pred = x_test.apply(lambda x: get_sentiment_label(sentiment_analyzer.polarity_scores(x)['compound'], threshold))
    accuracy = accuracy_score(y_test, y_pred)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_threshold = threshold

# Print the best threshold value
print("Best Threshold:", best_threshold)

# Apply the best threshold to sentiment analysis and create a new variable in the original data frame
df['sentiment_label_VADER'] = df['Text'].apply(lambda x: get_sentiment_label(sentiment_analyzer.polarity_scores(x)['compound'], best_threshold))

# Calculate accuracy
accuracy = accuracy_score(y, df['sentiment_label_VADER'])
print('Accuracy: {:.2f}%'.format(accuracy * 100))

df.head()

Best Threshold: 0.4
Accuracy: 82.13%


,Text,Sentiment,Text_Clean,sentiment_label_VADER
0,!....THE OWNERS REALLY REALLY need to quit bei...,0,owner really really need quit soooooo cheap le...,0
1,"(It wasn't busy either) Also, the building was...",0,busy either also building freezing cold,0
2,(The bathroom is just next door and very nice.),1,bathroom next door nice,0
3,* Both the Hot & Sour & the Egg Flower Soups w...,1,hot sour egg flower soup absolutely star,0
4,2 Thumbs Up!!,1,thumb,0


In [ ]:
sentiment_analyzer.polarity_scores("i love  pizza but its not good for")

{'neg': 0.29, 'neu': 0.467, 'pos': 0.243, 'compound': -0.1303}

##Naive Bayse Model




In [ ]:


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# In read_csv encoding cmd is fornon-ASCII characters
df =pd.read_excel("yelp_sentiment.xlsx")
df.head()
df.shape

,Text,Sentiment
0,!....THE OWNERS REALLY REALLY need to quit bei...,0
1,"(It wasn't busy either) Also, the building was...",0
2,(The bathroom is just next door and very nice.),1
3,* Both the Hot & Sour & the Egg Flower Soups w...,1
4,2 Thumbs Up!!,1


(996, 2)

In [ ]:
df.dtypes


Text         object
Sentiment     int64
dtype: object

In [ ]:


# Your DataFrame 'df' should already be defined here

# Now let's peform text cleaning on SMS messages
def clean_text(text):
# Convert the text to lowercase
  text = text.lower()
# Remove special characters and digits
  text = re.sub (r'[^a-z\s]', ' ', text)
# Remove URLs- r'http\S+|www.\S+' matches any website with http or www
  text = re.sub (r'http\S+|www.\S+', ' ', text)
# Remove Punctuations- r'[^\w\s]' matches any character that is not a word character
  text = re.sub (r'[^\w\s]', ' ', text)
# Remove Numbers- r'\d+' matches any digits (0-9)
  text = re.sub(r'\d+', ' ', text)
#Remove extra white spaces- r'\s+' matches any whitespaces, tabs, lines
  text = re.sub(r'\s+', ' ', text).strip()
# Remove @ mentions- It removes '@' followed by one or more alphanumeric characters
  text = re.sub(r'@\w+', ' ', text)
#Remove hashtags- It removes '#' followed by one or more alphanumeric characters
  text = re.sub (r'#[A-Za-z0-9]+', ' ', text)
  #Remove special characters- Removes anything which is not alphanumeric characters
  text = re.sub(r'[^\w\s]', ' ', text)
#Tokenize the text
  tokens = nltk.word_tokenize(text)
# Remove stop words
  stop_words = set (stopwords.words ('english'))
  tokens = [x for x in tokens if x not in stop_words]
# Lemmatize the text
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(x) for x in tokens]
# Join the tokens back into a string
  text = ' '.join(tokens)
  return text

# Apply the clean_text function to the 'Text' column and create a new 'Text_Clean' column
df['Text_Clean'] = df['Text'].apply(clean_text)
df.head()

,Text,Sentiment,Text_Clean
0,!....THE OWNERS REALLY REALLY need to quit bei...,0,owner really really need quit soooooo cheap le...
1,"(It wasn't busy either) Also, the building was...",0,busy either also building freezing cold
2,(The bathroom is just next door and very nice.),1,bathroom next door nice
3,* Both the Hot & Sour & the Egg Flower Soups w...,1,hot sour egg flower soup absolutely star
4,2 Thumbs Up!!,1,thumb


In [ ]:
# Export csv file to visualize in spreadsheet
df.to_csv('cleaned_data.csv')

In [ ]:
# Now let's begin with NB modeling by creatings features (x) and label (y)
x = df.Text_Clean
y= df.Sentiment
x.head()
y.head()
x.shape
print()
y.shape
df.info()

0    owner really really need quit soooooo cheap le...
1              busy either also building freezing cold
2                              bathroom next door nice
3             hot sour egg flower soup absolutely star
4                                                thumb
Name: Text_Clean, dtype: object

0    0
1    0
2    1
3    1
4    1
Name: Sentiment, dtype: int64

(996,)

(996,)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Text        996 non-null    object
 1   Sentiment   996 non-null    int64 
 2   Text_Clean  996 non-null    object
dtypes: int64(1), object(2)
memory usage: 23.5+ KB


In [ ]:
vectorizer = TfidfVectorizer()
x_vect = vectorizer.fit_transform(x)
print(x_vect.shape) # of features are 7881 below
print(type(x_vect))
print(vectorizer.get_feature_names_out() [0:10])# print the first 10 words/Features print (x_vect) # print each feature and the TFIDF score
print (x_vect)
print(vectorizer.get_feature_names_out()[1700])


(996, 1762)
<class 'scipy.sparse._csr.csr_matrix'>
['absolute' 'absolutely' 'absolutley' 'accident' 'accommodation'
 'accomodate' 'accountant' 'ache' 'acknowledged' 'across']
  (0, 1054)	0.19440790349330161
  (0, 1094)	0.27495402840294014
  (0, 1623)	0.24074830304134973
  (0, 1329)	0.23630410699281357
  (0, 608)	0.30419156527019303
  (0, 1744)	0.2870887025893978
  (0, 864)	0.27495402840294014
  (0, 243)	0.2870887025893978
  (0, 1435)	0.30419156527019303
  (0, 1221)	0.30419156527019303
  (0, 1012)	0.2457164915356873
  (0, 1243)	0.3693684395112203
  (0, 1082)	0.24074830304134973
  (1, 279)	0.3705080583643563
  (1, 610)	0.47695077183953116
  (1, 189)	0.4501346977349606
  (1, 25)	0.3024125551052469
  (1, 482)	0.39409731804820824
  (1, 198)	0.4311083903680505
  (2, 1021)	0.4062514412290358
  (2, 435)	0.5699008745858146
  (2, 1020)	0.47385051160518527
  (2, 111)	0.5344515434510028
  (3, 1462)	0.3108375717531069
  (3, 1)	0.34981770485824626
  :	:
  (989, 1696)	0.6097279864960563
  (989, 384)	

In [ ]:
#Spltting od data inot train and test
x_train, x_test, y_train, y_test = train_test_split(x_vect, y)
x_train.shape
x_test.shape
y_train.shape
y_test.shape

(747, 1762)

(249, 1762)

(747,)

(249,)

In [ ]:
# Build the NB model and make predictions on teh test data to get classification report
from sklearn.metrics import classification_report as cr
clf = MultinomialNB()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print (cr(y_test, y_pred))


MultinomialNB()

              precision    recall  f1-score   support

           0       0.81      0.74      0.77       129
           1       0.74      0.81      0.77       120

    accuracy                           0.77       249
   macro avg       0.77      0.77      0.77       249
weighted avg       0.77      0.77      0.77       249



In [ ]:
# Make predictions for the entire data and append into the original data
df['Predicted'] = clf.predict(x_vect)
df.sample(10)

,Text,Sentiment,Text_Clean,Predicted
388,"In the summer, you can dine in a charming outd...",1,summer dine charming outdoor patio delightful,1
692,The live music on Fridays totally blows.,0,live music friday totally blow,0
428,"I've had better, not only from dedicated boba ...",0,better dedicated boba tea spot even jenni pho,0
116,"Delicious NYC bagels, good selections of cream...",1,delicious nyc bagel good selection cream chees...,1
664,The food is very good for your typical bar food.,1,food good typical bar food,1
14,A great way to finish a great.,1,great way finish great,1
526,"Our server was very nice, and even though he l...",1,server nice even though looked little overwhel...,1
974,Will not be back!,0,back,0
757,The shrimp tender and moist.,1,shrimp tender moist,1
656,The first time I ever came here I had an amazi...,1,first time ever came amazing experience still ...,1


##Incresing the recall for 1 using threshod manipulation

In [ ]:
from sklearn.metrics import classification_report

# Build and train the MultinomialNB model
clf = MultinomialNB()
clf.fit(x_train, y_train)

# Make predictions on the test data
y_pred_prob = clf.predict_proba(x_test)[:, 1]

# Adjust the classification threshold (e.g., set it to 0.45)
threshold = 0.45
y_pred_adjusted = (y_pred_prob > threshold).astype(int)

# Get the classification report
report = classification_report(y_test, y_pred_adjusted, target_names=['Class 0', 'Class 1'])

# Print the classification report
print(report)


MultinomialNB()

              precision    recall  f1-score   support

     Class 0       0.87      0.63      0.73       129
     Class 1       0.69      0.90      0.78       120

    accuracy                           0.76       249
   macro avg       0.78      0.76      0.76       249
weighted avg       0.78      0.76      0.76       249

